# Document Chracterizzation


## Import of relevant libraries 

In [59]:
# Import of all the libraries used for the project
import os
import shutil
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from operator import itemgetter
import plotly.graph_objects as go
import spacy
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer


## Dataset import

In [2]:
# Defined a simple scraper  to go through all the link, visit the page and copy all the txt in the paragraph tag of the HTML file

def scraper(input):
    # create an empty array
    content = []

    # for loop on all the links in the dataframe
    for i in range(len(input["URL"])):
        url = input["URL"][i]
        # text = url
        text = ' '

        # check if actually the url is correct orr not
        if not (url.startswith('https://') or url.startswith('http://')):
            text += ' Link not good'
            content.append(text) 
            continue
        
        # Request the url and copy of the HTMl 
        page = requests.get(url)
        
        # Check if the response is positive or not, if not set that page was not found 
        if page.status_code != 200:
            text += ' Page not able to find'

        # Else take the text from all the p tag of the HTML content  
        else:
            soup = BeautifulSoup(page.content, 'html.parser')
            for p in soup.find_all('p'):
                text = text + ' ' + p.text
        
        content.append(text) 
    return content

In [3]:
## import the CSV file with the labels

data_scraper = pd.read_csv('Document Characterisation Data.csv', sep=',')
#print(len(data_scraper["URL"]))

# Calling the scraper function
data = scraper(data_scraper)

# Adding the scraped data into the Dataframe  
data_scraper['Content'] = data
data_scraper

,URL,Class,Content
0,https://issafrica.org/iss-today/time-to-tackle...,Analytical Report,"2020-11-25 On 26 September, hundreds of Mali..."
1,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By George Barros November 16, 2020 The Kreml..."
2,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By Ezgi Yazici November 13, 2020 Key Takeawa..."
3,http://www.understandingwar.org/backgrounder/w...,Analytical Report,By Katherine Lawlor with Brandon Wallace Key...
4,http://www.understandingwar.org/backgrounder/r...,Analytical Report,By Isabel Ivanescu and Ezgi Yazici Key Takea...
...,...,...,...
232,https://www.journalism.org/2019/03/26/for-loca...,Analytical Report,"Numbers, Facts and Trends Shaping Your World..."
233,https://www.journalism.org/2017/10/02/covering...,Analytical Report,"Numbers, Facts and Trends Shaping Your World..."
234,https://ctc.usma.edu/afghanistans-future-emira...,Analytical Report,Main Menu Seth G. Jones \nShare via:\n\n\n\n...
235,https://ctc.usma.edu/back-into-the-shadows-the...,Analytical Report,Main Menu Michael Knights \nShare via:\n\n\n...


In [4]:
# Function to load the files from local drive into Pandas data frame

def readFiles (src, fileClass):
    src_files = os.listdir(src)
    fileType = []
    content = []
    for i in range(len(src_files)):
        fileType.append(fileClass)
        glove_file = open(src + src_files[i], "r", encoding="utf8")
        glove_vectors_list = [word_and_vector.strip() for word_and_vector in glove_file.readlines()]
        content.append(" ".join(glove_vectors_list))
    
    columns = ["URL", "Class", "Content"]
    df = pd.DataFrame(columns=columns)

    df["URL"] = src_files
    df["Class"] = fileType
    df["Content"] = content

    return df


In [5]:
src = "./Data/Analytical Report/"
df_Analytical_Report = readFiles(src, "Analytical Report")

print(len(df_Analytical_Report['Content'][0]))

src = "./Data/Analytical Report2/"
df_Analytical_Report2 = readFiles(src, "Analytical Report")

print(len(df_Analytical_Report2['Content'][0]))

src = "./Data/Incident Report/"
df_Incident_Report = readFiles(src, "Incident Report")

src = "./Data/Situation Report/"
df_Situation_Report = readFiles(src, "Situation Report")

toConcat = [data_scraper, df_Analytical_Report, df_Analytical_Report2, df_Incident_Report, df_Situation_Report]

data_input = pd.concat(toConcat, ignore_index=True)
data_input

5725
5725


,URL,Class,Content
0,https://issafrica.org/iss-today/time-to-tackle...,Analytical Report,"2020-11-25 On 26 September, hundreds of Mali..."
1,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By George Barros November 16, 2020 The Kreml..."
2,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By Ezgi Yazici November 13, 2020 Key Takeawa..."
3,http://www.understandingwar.org/backgrounder/w...,Analytical Report,By Katherine Lawlor with Brandon Wallace Key...
4,http://www.understandingwar.org/backgrounder/r...,Analytical Report,By Isabel Ivanescu and Ezgi Yazici Key Takea...
...,...,...,...
413,int-rep-o-18-e.txt,Situation Report,Deputies are investigating after an explosive ...
414,int-rep-o-18-f.txt,Situation Report,Residents at a Springfield apartment complex w...
415,int-rep-o-18-g.txt,Situation Report,Several handheld explosive devices were found ...
416,int-rep-o-18-h.txt,Situation Report,Boulder police taped off the U.S. Bank at Broa...


## Experimental Data Analysis 

### Data frame informations and duplicates Duplicates

In [6]:
data_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   URL      418 non-null    object
 1   Class    418 non-null    object
 2   Content  418 non-null    object
dtypes: object(3)
memory usage: 9.9+ KB


In [7]:
# Number of items per class
print("Number of Analytical Report ", len(data_input[data_input['Class'] == 'Analytical Report']))
print("Number of Situation Report ", len(data_input[data_input['Class'] == 'Situation Report']))
print("Number of Incident Report ", len(data_input[data_input['Class'] == 'Incident Report']))
print("Number of Profile Report ", len(data_input[data_input['Class'] == 'Profile Report']))


Number of Analytical Report  131
Number of Situation Report  99
Number of Incident Report  88
Number of Profile Report  100


In [8]:
# Dropping the duplicate items 

data_input_dropped= data_input.drop_duplicates(keep="first").reset_index(drop=True)

In [9]:
# Number of items per class after dropping the duplicates
print("Number of Analytical Report ", len(data_input_dropped[data_input_dropped['Class'] == 'Analytical Report']))
print("Number of Situation Report ", len(data_input_dropped[data_input_dropped['Class'] == 'Situation Report']))
print("Number of Incident Report ", len(data_input_dropped[data_input_dropped['Class'] == 'Incident Report']))
print("Number of Profile Report ", len(data_input_dropped[data_input_dropped['Class'] == 'Profile Report']))


Number of Analytical Report  92
Number of Situation Report  98
Number of Incident Report  88
Number of Profile Report  98


In [10]:
data_input_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376 entries, 0 to 375
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   URL      376 non-null    object
 1   Class    376 non-null    object
 2   Content  376 non-null    object
dtypes: object(3)
memory usage: 8.9+ KB


### Pre Processing the data

In [11]:
# tockenizzation of the corpus 
def tokenize(corpus):

    # The backslash is an escape character to let the interpreter know we mean to use it as a string literal (- or ') 
    re_punctuation_string = '[\s,/.?\-\'`’]'  # split on spaces (\s),  commas (,), slash (/), fullstop (.), question marks (?), hyphens (-), and apostrophe (').
    tokenized = re.split(re_punctuation_string, corpus) # in python's regex, [...] is an alternative to writing .|.|.
    tokenized = list(filter(None, tokenized)) # remove empty strings from list 
    tokenized = [token.lower() for token in tokenized] # Lowercasing everything
    tokenized = ['<Number>' if token.isdigit() else token for token in tokenized ]  # converting all the numbers to <Number>


    return tokenized    



In [12]:
# function to count the tockenization
def distinct_tokens_count(tokenized_corpus):

    distinct_tokens_count = {}
    for t_review in tokenized_corpus:

        if t_review not in distinct_tokens_count.keys():
            distinct_tokens_count[t_review] = 1
        else:
            distinct_tokens_count[t_review] += 1

    # Sort the key value to greter to smaller
    distinct_tokens_count = sorted(distinct_tokens_count.items(), key=itemgetter(1), reverse=True) 

    #for token, count in distinct_tokens_count:
        #print("{:<14s} {:<10d}".format(token, count))
    return distinct_tokens_count



In [13]:
# function to pre process the text and count the number of words
def process(dataframe):

    tokenized = []
    tokens_count = []
    class_type = []
    for i in range(dataframe.shape[0]):
        #print(dataframe['Content'][i])
        #print(i)
        
        # tokenizing the corpus
        token = tokenize(dataframe['Content'][i])
        tokenized.append(token)
        # counting the tokens
        tokens_count.append(distinct_tokens_count(token))

        if dataframe['Class'][i] == 'Analytical Report':
            class_type.append(0)

        elif dataframe['Class'][i] == 'Incident Report':
            class_type.append(1)

        elif dataframe['Class'][i] == 'Profile Report':
            class_type.append(2)

        elif dataframe['Class'][i] == 'Situation Report':
            class_type.append(3)


    dataframe['Tokenized'] = tokenized
    dataframe['Tokens Count'] = tokens_count
    dataframe['Class Type'] = class_type

    return dataframe

In [14]:
data_processed = process(data_input_dropped)      

data_processed

,URL,Class,Content,Tokenized,Tokens Count,Class Type
0,https://issafrica.org/iss-today/time-to-tackle...,Analytical Report,"2020-11-25 On 26 September, hundreds of Mali...","[<Number>, <Number>, <Number>, on, <Number>, s...","[(the, 127), (and, 83), (women, 82), (to, 70),...",0
1,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By George Barros November 16, 2020 The Kreml...","[by, george, barros, november, <Number>, <Numb...","[(the, 67), (<Number>, 50), (to, 44), (csto, 3...",0
2,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By Ezgi Yazici November 13, 2020 Key Takeawa...","[by, ezgi, yazici, november, <Number>, <Number...","[(the, 41), (in, 25), (to, 24), (s, 22), (and,...",0
3,http://www.understandingwar.org/backgrounder/w...,Analytical Report,By Katherine Lawlor with Brandon Wallace Key...,"[by, katherine, lawlor, with, brandon, wallace...","[(s, 131), (the, 121), (to, 120), (<Number>, 1...",0
4,http://www.understandingwar.org/backgrounder/r...,Analytical Report,By Isabel Ivanescu and Ezgi Yazici Key Takea...,"[by, isabel, ivanescu, and, ezgi, yazici, key,...","[(in, 42), (the, 37), (and, 34), (a, 23), (to,...",0
...,...,...,...,...,...,...
371,int-rep-o-18-e.txt,Situation Report,Deputies are investigating after an explosive ...,"[deputies, are, investigating, after, an, expl...","[(the, 52), (a, 43), (was, 35), (to, 25), (dev...",3
372,int-rep-o-18-f.txt,Situation Report,Residents at a Springfield apartment complex w...,"[residents, at, a, springfield, apartment, com...","[(the, 64), (a, 47), (was, 27), (in, 22), (bom...",3
373,int-rep-o-18-g.txt,Situation Report,Several handheld explosive devices were found ...,"[several, handheld, explosive, devices, were, ...","[(the, 82), (a, 67), (bomb, 30), (to, 27), (wa...",3
374,int-rep-o-18-h.txt,Situation Report,Boulder police taped off the U.S. Bank at Broa...,"[boulder, police, taped, off, the, u, s, bank,...","[(the, 83), (a, 77), (was, 42), (to, 36), (bom...",3


### Feature extraction

In [15]:
def adding_features(dataframe):
    length = []
    unique_tokens = []
    tokens_percentage = []
    
    for i in range(dataframe.shape[0]):
        leng = len(data_processed['Tokenized'][i])
        length.append(leng)
        unique_tokens.append(len(data_processed['Tokens Count'][i]))

        distinct_tokens_percentage = {}
        for token, count in data_processed['Tokens Count'][i]:
            distinct_tokens_percentage[token] = (count * 100) / leng
        
        distinct_tokens_percentage = sorted(distinct_tokens_percentage.items(), key=itemgetter(1), reverse=True)
        tokens_percentage.append(distinct_tokens_percentage)
    
    
    dataframe['Length'] = length
    dataframe['Unique Tokens'] = unique_tokens
    dataframe['Tokens Percentage'] = tokens_percentage
    # tahen out all the page not accessible with message "Page not found"
    dataframe = dataframe[dataframe['Length'] > 10].reset_index(drop=True)

    return dataframe

In [16]:
data_add_features = adding_features(data_processed)

data_Analytical = data_add_features[data_add_features['Class'] == 'Analytical Report']
data_Incident = data_add_features[data_add_features['Class'] == 'Incident Report']
data_Profile = data_add_features[data_add_features['Class'] == 'Profile Report']
data_Situation = data_add_features[data_add_features['Class'] == 'Situation Report']

data_add_features

,URL,Class,Content,Tokenized,Tokens Count,Class Type,Length,Unique Tokens,Tokens Percentage
0,https://issafrica.org/iss-today/time-to-tackle...,Analytical Report,"2020-11-25 On 26 September, hundreds of Mali...","[<Number>, <Number>, <Number>, on, <Number>, s...","[(the, 127), (and, 83), (women, 82), (to, 70),...",0,2313,521,"[(the, 5.490704712494596), (and, 3.58841331603..."
1,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By George Barros November 16, 2020 The Kreml...","[by, george, barros, november, <Number>, <Numb...","[(the, 67), (<Number>, 50), (to, 44), (csto, 3...",0,1393,533,"[(the, 4.809763101220388), (<Number>, 3.589375..."
2,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By Ezgi Yazici November 13, 2020 Key Takeawa...","[by, ezgi, yazici, november, <Number>, <Number...","[(the, 41), (in, 25), (to, 24), (s, 22), (and,...",0,824,366,"[(the, 4.975728155339806), (in, 3.033980582524..."
3,http://www.understandingwar.org/backgrounder/w...,Analytical Report,By Katherine Lawlor with Brandon Wallace Key...,"[by, katherine, lawlor, with, brandon, wallace...","[(s, 131), (the, 121), (to, 120), (<Number>, 1...",0,3528,985,"[(s, 3.7131519274376417), (the, 3.429705215419..."
4,http://www.understandingwar.org/backgrounder/r...,Analytical Report,By Isabel Ivanescu and Ezgi Yazici Key Takea...,"[by, isabel, ivanescu, and, ezgi, yazici, key,...","[(in, 42), (the, 37), (and, 34), (a, 23), (to,...",0,878,390,"[(in, 4.783599088838269), (the, 4.214123006833..."
...,...,...,...,...,...,...,...,...,...
350,int-rep-o-18-e.txt,Situation Report,Deputies are investigating after an explosive ...,"[deputies, are, investigating, after, an, expl...","[(the, 52), (a, 43), (was, 35), (to, 25), (dev...",3,784,313,"[(the, 6.63265306122449), (a, 5.48469387755102..."
351,int-rep-o-18-f.txt,Situation Report,Residents at a Springfield apartment complex w...,"[residents, at, a, springfield, apartment, com...","[(the, 64), (a, 47), (was, 27), (in, 22), (bom...",3,791,331,"[(the, 8.09102402022756), (a, 5.94184576485461..."
352,int-rep-o-18-g.txt,Situation Report,Several handheld explosive devices were found ...,"[several, handheld, explosive, devices, were, ...","[(the, 82), (a, 67), (bomb, 30), (to, 27), (wa...",3,941,367,"[(the, 8.71413390010627), (a, 7.12008501594048..."
353,int-rep-o-18-h.txt,Situation Report,Boulder police taped off the U.S. Bank at Broa...,"[boulder, police, taped, off, the, u, s, bank,...","[(the, 83), (a, 77), (was, 42), (to, 36), (bom...",3,1355,488,"[(the, 6.125461254612546), (a, 5.6826568265682..."


In [17]:
# Function to plot the violin plot for the dataframe   
def plot_violin(df, to_be_plot):

    fig = go.Figure()

    reports = ['Analytical Report', 'Incident Report', 'Profile Report', 'Situation Report']

    for report in reports:
        fig.add_trace(go.Violin(x=df['Class'][df['Class'] == report],
                                y=df[to_be_plot][df['Class'] == report],
                                name=report,
                                box_visible=True,
                                meanline_visible=True))
    
    fig.update_layout(
    title="Violin plot of the " + to_be_plot + " for type of reports",
    yaxis_range=[0, 5000],
    )
    fig.show()

    return


In [18]:
plot_violin(data_add_features, "Length")
plot_violin(data_add_features, "Unique Tokens")

In [19]:
# function to tag the part of speech using the pre trained model from Spacy

def part_Speech_tagging(data_in):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(data_in)

    columns_tok = ["token_text", "token_lemma", "token_pos", "token_dep"]
    df_tok = pd.DataFrame(columns=columns_tok)
    text = []
    lemma = []
    pos = []
    dep =[]
    for token in doc:
        text.append(token.text)
        lemma.append(token.lemma_)
        pos.append(token.pos_)
        dep.append(token.dep_)

    df_tok["token_text"] = text
    df_tok["token_lemma"] = lemma
    df_tok["token_pos"] = pos
    df_tok["token_dep"] = dep

    return df_tok


In [20]:
# Function to add feature based on the part to speech tagging

def feature_add_speech(data):
    # counting the root of a phrase by different type might give me better some understanding of different type of the documentes
    root_verb = []
    root_noun = []
    root_propn = []
    root_aux = []
    # also the counting the type of tags used in the phrases of the document can give some insights of the type of document
    adj = []
    adv =[]
    noun = []
    num = []
    pron = []
    propn = []
    verb = []
    
    for i in range(data.shape[0]):
        #print(i)
        length = data['Length'][i]
        df_tagged = part_Speech_tagging(data['Content'][i])

        a = df_tagged[df_tagged["token_dep"] == 'ROOT']
        if 'VERB' in a['token_pos'].value_counts():
            root_verb.append(a['token_pos'].value_counts()['VERB'] * 100 / length)
        else:
            root_verb.append(0)
        
        if 'NOUN' in a['token_pos'].value_counts():
            root_noun.append(a['token_pos'].value_counts()['NOUN'] * 100 / length)
        else:
            root_noun.append(0)

        if 'AUX' in a['token_pos'].value_counts():
            root_aux.append(a['token_pos'].value_counts()['AUX'] * 100 / length)
        else:
            root_aux.append(0)

        if 'PROPN' in a['token_pos'].value_counts():
            root_propn.append(a['token_pos'].value_counts()['PROPN'] * 100 / length)
        else:
            root_propn.append(0)
        

        b = df_tagged['token_pos'].value_counts()

        if 'ADJ' in b:
            adj.append(b['ADJ'] * 100 / length)
        else:
            adj.append(0)
        
        if 'ADV' in b:
            adv.append(b['ADV'] * 100 / length)
        else:
            adv.append(0)

        if 'NOUN' in b:
            noun.append(b['NOUN'] * 100 / length)
        else:
            noun.append(0)

        if 'NUM' in b:
            num.append(b['NUM'] * 100 / length)
        else:
            num.append(0)

        if 'PRON' in b:
            pron.append(b['PRON'] * 100 / length)
        else:
            pron.append(0)

        if 'PROPN' in b:
            propn.append(b['PROPN'] * 100 / length)
        else:
            propn.append(0)

        if 'VERB' in b:
            verb.append(b['VERB'] * 100 / length)
        else:
            verb.append(0)
        

    
    data['Root_verb'] = root_verb
    data['Root_noun'] = root_noun
    data['Root_aux'] = root_aux
    data['Root_propn'] = root_propn

    data['Adj'] = adj
    data['Adv'] = adv
    data['Noun'] = noun
    data['Num'] = num
    data['Pron'] = pron
    data['Propn'] = propn
    data['Verb'] = verb

    return data



data_add_speech_tag = feature_add_speech(data_add_features)
data_add_speech_tag

,URL,Class,Content,Tokenized,Tokens Count,Class Type,Length,Unique Tokens,Tokens Percentage,Root_verb,Root_noun,Root_aux,Root_propn,Adj,Adv,Noun,Num,Pron,Propn,Verb
0,https://issafrica.org/iss-today/time-to-tackle...,Analytical Report,"2020-11-25 On 26 September, hundreds of Mali...","[<Number>, <Number>, <Number>, on, <Number>, s...","[(the, 127), (and, 83), (women, 82), (to, 70),...",0,2313,521,"[(the, 5.490704712494596), (and, 3.58841331603...",4.020752,0.172936,0.907912,0.172936,10.722006,3.112840,27.842629,2.204929,1.080847,6.009511,13.186338
1,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By George Barros November 16, 2020 The Kreml...","[by, george, barros, november, <Number>, <Numb...","[(the, 67), (<Number>, 50), (to, 44), (csto, 3...",0,1393,533,"[(the, 4.809763101220388), (<Number>, 3.589375...",2.153625,0.789663,0.143575,0.287150,7.753051,2.153625,20.459440,3.445800,0.430725,18.808327,10.050251
2,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By Ezgi Yazici November 13, 2020 Key Takeawa...","[by, ezgi, yazici, november, <Number>, <Number...","[(the, 41), (in, 25), (to, 24), (s, 22), (and,...",0,824,366,"[(the, 4.975728155339806), (in, 3.033980582524...",2.669903,0.364078,0.606796,0.364078,10.800971,3.276699,18.689320,1.699029,1.092233,16.626214,10.679612
3,http://www.understandingwar.org/backgrounder/w...,Analytical Report,By Katherine Lawlor with Brandon Wallace Key...,"[by, katherine, lawlor, with, brandon, wallace...","[(s, 131), (the, 121), (to, 120), (<Number>, 1...",0,3528,985,"[(s, 3.7131519274376417), (the, 3.429705215419...",2.097506,0.255102,0.793651,0.623583,9.325397,3.032880,15.731293,3.713152,0.963719,18.764172,8.928571
4,http://www.understandingwar.org/backgrounder/r...,Analytical Report,By Isabel Ivanescu and Ezgi Yazici Key Takea...,"[by, isabel, ivanescu, and, ezgi, yazici, key,...","[(in, 42), (the, 37), (and, 34), (a, 23), (to,...",0,878,390,"[(in, 4.783599088838269), (the, 4.214123006833...",2.847380,0.227790,0.683371,0.113895,9.794989,2.847380,17.539863,2.050114,0.113895,15.034169,11.389522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,int-rep-o-18-e.txt,Situation Report,Deputies are investigating after an explosive ...,"[deputies, are, investigating, after, an, expl...","[(the, 52), (a, 43), (was, 35), (to, 25), (dev...",3,784,313,"[(the, 6.63265306122449), (a, 5.48469387755102...",4.846939,0.382653,0.510204,0.000000,5.484694,1.913265,23.341837,1.020408,2.806122,11.096939,13.520408
351,int-rep-o-18-f.txt,Situation Report,Residents at a Springfield apartment complex w...,"[residents, at, a, springfield, apartment, com...","[(the, 64), (a, 47), (was, 27), (in, 22), (bom...",3,791,331,"[(the, 8.09102402022756), (a, 5.94184576485461...",5.689001,0.000000,0.505689,0.000000,5.941846,1.011378,26.295828,0.379267,2.022756,8.975980,13.653603
352,int-rep-o-18-g.txt,Situation Report,Several handheld explosive devices were found ...,"[several, handheld, explosive, devices, were, ...","[(the, 82), (a, 67), (bomb, 30), (to, 27), (wa...",3,941,367,"[(the, 8.71413390010627), (a, 7.12008501594048...",5.951116,0.212540,0.425080,0.000000,5.738576,1.062699,26.248672,0.318810,1.275239,11.902232,12.964931
353,int-rep-o-18-h.txt,Situation Report,Boulder police taped off the U.S. Bank at Broa...,"[boulder, police, taped, off, the, u, s, bank,...","[(the, 83), (a, 77), (was, 42), (to, 36), (bom...",3,1355,488,"[(the, 6.125461254612546), (a, 5.6826568265682...",5.461255,0.000000,0.442804,0.000000,5.018450,2.583026,23.690037,0.738007,3.025830,11.734317,14.169742


In [112]:
# Function to remove the stopwords and symbols from the
def remove_stopwords(corpus):
    stopword = stopwords.words('english')
    symbols = "!\"#$%&()'*+-–./:;<=>?@[\]^_`{|}~\n"
    copy_corpus = []
    for token in corpus:
        if token in stopword:
             copy_corpus.append("")
        elif token in symbols:
             copy_corpus.append("")
        else:
            copy_corpus.append(token)
    return copy_corpus

# function to get right tag for the lemmatizzation 
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# Function to lemmatize the articles
def lemmatizzation(corpus):
    lemmatizer = WordNetLemmatizer()
    
    lemmatized_list = []

    for token in corpus:
        lemmatized = lemmatizer.lemmatize(token, get_wordnet_pos(token))

        lemmatized_list.append(lemmatized)

    return lemmatized_list

# pre processing before the feature extraction with TF-IDF 
def pre_extraction_TFIDF(in_data):
    out = []
    for i in range(in_data.shape[0]):
        corpus = in_data['Content'][i]
        # The backslash is an escape character to let the interpreter know we mean to use it as a string literal (- or ') 
        re_punctuation_string = '[\s,/.?\-\'`’]'  # split on spaces (\s),  commas (,), slash (/), fullstop (.), question marks (?), hyphens (-), and apostrophe (').
        tokenized = re.split(re_punctuation_string, corpus) # in python's regex, [...] is an alternative to writing .|.|.
        tokenized = [token.lower() for token in tokenized] # Lowercasing everything
        tokenized = remove_stopwords(tokenized) # remove all the stop words and the symbols 
        tokenized = list(filter(None, tokenized)) # remove empty strings from list 

        tokenized = ['<Number>' if token.isdigit() else token for token in tokenized ]  # converting all the numbers to <Number>

        lemmatized = lemmatizzation(tokenized)

        out.append(lemmatized)
    
    in_data['Pre TFIDF'] = out

    return in_data

In [113]:
# feature extraction with TF-IDF

def feature_extraction_TFIDF(in_data):

    out_data = pre_extraction_TFIDF(in_data)
    vectorizer = TfidfVectorizer(min_df=0.01)
    out_data['Pre TFIDF'] = [" ".join(doc) for doc in out_data['Pre TFIDF'].values]
    vectors = vectorizer.fit_transform(out_data['Pre TFIDF'])

    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()
    denselist = dense.tolist()

    df = pd.DataFrame(denselist, columns=feature_names)

    return df

In [121]:
# function to extract all the features from the dataset
def data_features(df_in):
    features_TFIDF = feature_extraction_TFIDF(df_in)
    print(features_TFIDF.shape)

    feature_columns = [ 'Class Type','Length', 'Unique Tokens',  'Root_verb',
       'Root_noun', 'Root_aux', 'Root_propn', 'Adj', 'Adv', 'Noun', 'Num',
       'Pron', 'Propn', 'Verb']
    
    original_features = df_in[feature_columns]
    #print(original_features.shape)

    feature_df = pd.concat([original_features, features_TFIDF], axis=1)
    #print(feature_df.shape)

    return feature_df


In [122]:
df_features = data_features(data_add_speech_tag)
df_features

(355, 5310)
(355, 14)
(355, 5324)


,Class Type,Length,Unique Tokens,Root_verb,Root_noun,Root_aux,Root_propn,Adj,Adv,Noun,...,youth,youtube,zachary,zakir,zawahiri,zealand,zero,zionist,zone,zubair
0,0,2313,521,4.020752,0.172936,0.907912,0.172936,10.722006,3.112840,27.842629,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0,1393,533,2.153625,0.789663,0.143575,0.287150,7.753051,2.153625,20.459440,...,0.0,0.018055,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0,824,366,2.669903,0.364078,0.606796,0.364078,10.800971,3.276699,18.689320,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.016871,0.0
3,0,3528,985,2.097506,0.255102,0.793651,0.623583,9.325397,3.032880,15.731293,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0,878,390,2.847380,0.227790,0.683371,0.113895,9.794989,2.847380,17.539863,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,3,784,313,4.846939,0.382653,0.510204,0.000000,5.484694,1.913265,23.341837,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
351,3,791,331,5.689001,0.000000,0.505689,0.000000,5.941846,1.011378,26.295828,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
352,3,941,367,5.951116,0.212540,0.425080,0.000000,5.738576,1.062699,26.248672,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
353,3,1355,488,5.461255,0.000000,0.442804,0.000000,5.018450,2.583026,23.690037,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


Pre trained BERT models

https://pypi.org/project/pytorch-pretrained-bert/

https://pytorch.org/hub/huggingface_pytorch-transformers/

https://medium.com/@aniruddha.choudhury94/part-2-bert-fine-tuning-tutorial-with-pytorch-for-text-classification-on-the-corpus-of-linguistic-18057ce330e1


Spacy linguistic features
https://spacy.io/usage/linguistic-features

TF-IDF https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089

